### This example demonstrates the insertion of a human-defined classifier into the XCSF population set.

In [1]:
import json
import numpy as np
from typing import Final

import xcsf

RANDOM_STATE = 1
np.random.seed(RANDOM_STATE)

### Initialise XCSF with an empty population

In [2]:
xcs = xcsf.XCS(
    x_dim=8,
    y_dim=1,
    n_actions=1,
    random_state=RANDOM_STATE,
    max_trials=1,
    pop_init=False,  # start with an empty population set
    action={"type": "integer"},
    condition={"type": "hyperrectangle_csr"},
    prediction={"type": "nlms_linear"},
)

print(json.dumps(xcs.get_params(), indent=4))

{
    "version": "1.3.0",
    "x_dim": 8,
    "y_dim": 1,
    "n_actions": 1,
    "omp_num_threads": 8,
    "random_state": 1,
    "pop_init": false,
    "max_trials": 1,
    "perf_trials": 1000,
    "pop_size": 2000,
    "loss_func": "mae",
    "set_subsumption": false,
    "theta_sub": 100,
    "e0": 0.01,
    "alpha": 0.1,
    "nu": 5,
    "beta": 0.1,
    "delta": 0.1,
    "theta_del": 20,
    "init_fitness": 0.01,
    "init_error": 0,
    "m_probation": 10000,
    "stateful": true,
    "compaction": false,
    "ea": {
        "select_type": "roulette",
        "theta_ea": 50,
        "lambda": 2,
        "p_crossover": 0.8,
        "err_reduc": 1,
        "fit_reduc": 0.1,
        "subsumption": false,
        "pred_reset": false
    },
    "condition": {
        "type": "hyperrectangle_csr"
    },
    "prediction": {
        "type": "nlms_linear"
    },
    "action": {
        "type": "integer"
    }
}


### Add a classifier to the population set

In [3]:
classifier = {
    "error": 10,  # each of these properties are optional
    "fitness": 1.01,
    "accuracy": 2,
    "set_size": 100,
    "numerosity": 2,
    "experience": 3,
    "time": 3,
    "samples_seen": 2,
    "samples_matched": 1,
    "condition": {
        "type": "hyperrectangle_csr",
        "center": [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8],
        "spread": [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8],
        "mutation": [0.2],  # this parameter still self-adapts
    },
    "action": {"type": "integer", "action": 0, "mutation": [0.28]}
    # prediction is absent and therefore initialised as normal
}

json_str = json.dumps(classifier)  # dictionary to JSON

# The json_insert_cl() function can be used to insert a single new classifier
# into the population. The new classifier is initialised with a random
# condition, action, prediction, and then any supplied properties overwrite
# these values. This means that all properties are optional. If the population
# set numerosity exceeds xcs.POP_SIZE after inserting the rule, the standard
# roulette wheel deletion mechanism will be invoked to maintain the population
# limit.
xcs.json_insert_cl(json_str)  # insert in [P]

### Population before fitting

In [4]:
pop_str: str = xcs.json()
print(pop_str)

{
	"classifiers":	[{
			"error":	10,
			"fitness":	1.01,
			"accuracy":	2.2204460492503131e-16,
			"set_size":	100,
			"numerosity":	2,
			"experience":	3,
			"time":	3,
			"samples_seen":	2,
			"samples_matched":	1,
			"current_match":	false,
			"current_action":	0,
			"current_prediction":	[0],
			"condition":	{
				"type":	"hyperrectangle_csr",
				"center":	[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8],
				"spread":	[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8],
				"mutation":	[0.2]
			},
			"action":	{
				"type":	"integer",
				"action":	0,
				"mutation":	[0.28]
			},
			"prediction":	{
				"type":	"nlms_linear",
				"weights":	[1, 0, 0, 0, 0, 0, 0, 0, 0],
				"eta":	0.010284224009722742,
				"mutation":	[0.90494135470679848]
			}
		}]
}


### Fit a new sample

In [5]:
X = np.asarray([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8]).reshape(1, -1)
y = np.random.random(1)

xcs.fit(X, y, warm_start=True)  # use existing population

time=00:00:00.000 trials=0 train=0.58298 pset=1 mset=0.1 mfrac=0.00
time=00:00:00.000


### Population after fitting

In [6]:
pop_str: str = xcs.json()
print(pop_str)

{
	"classifiers":	[{
			"error":	7.6457444988243566,
			"fitness":	1.009,
			"accuracy":	3.8273757017989739e-16,
			"set_size":	75.5,
			"numerosity":	2,
			"experience":	4,
			"time":	3,
			"samples_seen":	3,
			"samples_matched":	2,
			"current_match":	true,
			"current_action":	0,
			"current_prediction":	[1],
			"condition":	{
				"type":	"hyperrectangle_csr",
				"center":	[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8],
				"spread":	[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8],
				"mutation":	[0.2]
			},
			"action":	{
				"type":	"integer",
				"action":	0,
				"mutation":	[0.28]
			},
			"prediction":	{
				"type":	"nlms_linear",
				"weights":	[0.99802780384987577, -0.00019721961501242832, -0.00039443923002485665, -0.000591658845037285, -0.0007888784600497133, -0.00098609807506214154, -0.00118331769007457, -0.001380537305086998, -0.0015777569200994266],
				"eta":	0.010284224009722742,
				"mutation":	[0.90494135470679848]
			}
		}]
}
